In [ ]:
!pip install transformers

In [ ]:
import pandas as pd
import os
import cv2
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.utils import resample
import tensorflow as tf
import tqdm as tqdm
from transformers import AutoImageProcessor, TFViTModel, ViTForImageClassification, TFSegformerForSemanticSegmentation
from tensorflow.keras.utils import to_categorical
from tensorflow.keras import Sequential, Model
from tensorflow.keras.layers import Dense, Input, concatenate, Flatten, Conv2D, Dropout,GlobalAveragePooling2D
from sklearn.metrics import accuracy_score
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder

from tensorflow.keras.callbacks import LearningRateScheduler, EarlyStopping
from tensorflow.keras.applications import ResNet50
from sklearn.metrics import classification_report

In [ ]:
folder = '/content/drive/MyDrive/Proyecto Cultivos/deepglobe/tiles-128/'

In [ ]:
class_to_rgb = {
    'urban_land': (0, 255, 255),
    'agriculture_land': (255, 255, 0),
    'rangeland': (255, 0, 255),
    'forest_land': (0, 255, 0),
    'water': (0, 0, 255),
    'barren_land': (255, 255, 255),
    'unknown': (0, 0, 0)
}

In [ ]:
# Load the data
loaded_data = np.load('/content/drive/MyDrive/Proyecto Cultivos/deepglobe/datacompressed-128_small.npz')
df_small = pd.read_csv('/content/drive/MyDrive/Proyecto Cultivos/deepglobe/df-128_small.csv')
# Access images and masks from the loaded data
loaded_images = loaded_data['images']

In [ ]:
y = df_small['label'].values

In [ ]:
# Transformamos las etiquetas en categóricas para poder pasarlas al modelo

label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(y)
y_one_hot = to_categorical(y_encoded, num_classes=7)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(loaded_images, y_one_hot, test_size=0.15, random_state=42)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.15, random_state=42)

Extraemos los embeddings. Utilizaremos ViT de Google para ello:

In [ ]:
image_processor = AutoImageProcessor.from_pretrained("google/vit-base-patch16-224-in21k")
modelViTClasif = TFViTModel.from_pretrained("google/vit-base-patch16-224-in21k")

All model checkpoint layers were used when initializing TFViTModel.

All the layers of TFViTModel were initialized from the model checkpoint at google/vit-base-patch16-224-in21k.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFViTModel for predictions without further training.


In [ ]:
# Importante asegurarse en la extracción de embeddings que si algún valor resulta null, nan o inf, se debe descartar
# (podría darse alguna imagen con muy poca información y que son pequeñas)
# También descartar en ese caso su respectiva etiqueta.
from tqdm import tqdm

embeddings_train = []
filtered_labels_train = []
for image, label in tqdm(zip(X_train, y_train), desc="Processing..."):
    if np.isnan(image).any() or np.isinf(image).any():
        print(f"Skipping image due to NaN or Infinity values.")
        continue
    inputs = image_processor(image, return_tensors='tf', do_rescale=False)
    outputs = modelViTClasif(**inputs)
    last_hidden_states = outputs.last_hidden_state
    embedding = tf.reduce_mean(last_hidden_states, axis=1)
    if np.isnan(embedding).any() or np.isinf(embedding).any():
        print(f"Skipping image due to NaN or Infinity values in embedding.")
        continue
    embeddings_train.append(embedding.numpy())
    filtered_labels_train.append(label)

X_emb_img_train = np.vstack(embeddings_train)
y_train_filtered = np.array(filtered_labels_train)

Processing...: 19747it [2:21:14,  2.24it/s]C:\Users\Conaf\AppData\Roaming\Python\Python310\site-packages\transformers\image_transforms.py:394: RuntimeWarning: divide by zero encountered in divide
  image = (image - mean) / std
Processing...: 19748it [2:21:14,  2.19it/s]

Skipping image due to NaN or Infinity values in embedding.


Processing...: 31070it [3:42:52,  2.32it/s]


In [ ]:
# Guardamos los embeddings y las labels para utilizarlas después
np.save('/content/drive/MyDrive/Proyecto Cultivos/deepglobe/embeddings_train_small.npy', X_emb_img_train)
np.save('/content/drive/MyDrive/Proyecto Cultivos/deepglobe/y_train_filtered.npy', y_train_filtered)

In [ ]:
# Se repite el procedimiento con las muestras de test y validación
from tqdm import tqdm

embeddings_val = []
filtered_labels_val = []
for image, label in tqdm(zip(X_val, y_val), desc="Processing..."):
    if np.isnan(image).any() or np.isinf(image).any():
        print(f"Skipping image due to NaN or Infinity values.")
        continue
    inputs = image_processor(image, return_tensors='tf', do_rescale=False)
    outputs = modelViTClasif(**inputs)
    last_hidden_states = outputs.last_hidden_state
    embedding = tf.reduce_mean(last_hidden_states, axis=1)
    embeddings_val.append(embedding.numpy())
    filtered_labels_val.append(label)

# Reconvertir los embeddings al formato para entrenar
X_emb_img_val = np.vstack(embeddings_val)
y_val_filtered = np.array(filtered_labels_val)

np.save('/content/drive/MyDrive/Proyecto Cultivos/deepglobe/embeddings_val_small.npy', X_emb_img_val)
np.save('/content/drive/MyDrive/Proyecto Cultivos/deepglobe/y_val_filtered.npy', y_val_filtered)

Processing...: 5484it [39:24,  2.32it/s]


In [ ]:
from tqdm import tqdm

embeddings_test = []
filtered_labels_test = []
for image, label in tqdm(zip(X_test, y_test), desc="Processing..."):
    if np.isnan(image).any() or np.isinf(image).any():
        print(f"Skipping image due to NaN or Infinity values.")
        continue
    inputs = image_processor(image, return_tensors='tf', do_rescale=False)
    outputs = modelViTClasif(**inputs)
    last_hidden_states = outputs.last_hidden_state
    embedding = tf.reduce_mean(last_hidden_states, axis=1)
    embeddings_test.append(embedding.numpy())
    filtered_labels_test.append(label)

# Reconvertir los embeddings al formato para entrenar
X_emb_img_test = np.vstack(embeddings_test)
y_test_filtered = np.array(filtered_labels_test)

np.save('/content/drive/MyDrive/Proyecto Cultivos/deepglobe/embeddings_test_small.npy', X_emb_img_test)
np.save('/content/drive/MyDrive/Proyecto Cultivos/deepglobe/y_test_filtered.npy', y_test_filtered)

Processing...: 6451it [44:46,  2.40it/s]


In [ ]:
# Cargamos embeddings y labels

embeddings_train = np.load('/content/drive/MyDrive/Proyecto Cultivos/deepglobe/embeddings_train_small.npy')
y_train_filtered = np.load('/content/drive/MyDrive/Proyecto Cultivos/deepglobe/y_train_filtered.npy')
embeddings_val = np.load('/content/drive/MyDrive/Proyecto Cultivos/deepglobe/embeddings_val_small.npy')
y_val_filtered = np.load('/content/drive/MyDrive/Proyecto Cultivos/deepglobe/y_val_filtered.npy')
embeddings_test = np.load('/content/drive/MyDrive/Proyecto Cultivos/deepglobe/embeddings_test_small.npy')
y_test_filtered = np.load('/content/drive/MyDrive/Proyecto Cultivos/deepglobe/y_test_filtered.npy')

In [ ]:
print(f"embeddings_train: {embeddings_train.shape}")
print(f"y_train_filtered: {y_train_filtered.shape}")
print(f"embeddings_val: {embeddings_val.shape}")
print(f"y_val_filtered: {y_val_filtered.shape}")
print(f"embeddings_test: {embeddings_test.shape}")
print(f"y_test_filtered: {y_test_filtered.shape}")

embeddings_train: (31069, 768)
y_train_filtered: (31069, 7)
embeddings_val: (5484, 768)
y_val_filtered: (5484, 7)
embeddings_test: (6451, 768)
y_test_filtered: (6451, 7)


In [ ]:
# Hacemos GridSearch para buscar unos buenos parámetros

inputs = Input(shape=(embeddings_train.shape[1],))
x = Dense(128, activation='relu')(inputs)
predictions = Dense(7, activation='softmax')(x)

modelViT = Model(inputs, predictions)


param_dist = {
    'learning_rate': np.random.uniform(0.0001, 0.1, size=10),
    'batch_size': [16, 32, 64],
    'num_hidden_units': [64, 128, 256],
    'dropout_rate': np.random.uniform(0.2, 0.5, size=10)
}

num_trials = 20
epochs = 100
best_hyperparameters = None
best_validation_accuracy = 0.0

for _ in range(num_trials):
  sampled_params = {
      'learning_rate': np.random.choice(param_dist['learning_rate']),
      'batch_size': np.random.choice(param_dist['batch_size']),
      'num_hidden_units': np.random.choice(param_dist['num_hidden_units']),
      'dropout_rate': np.random.choice(param_dist['dropout_rate'])
  }

  modelViT.compile(optimizer=Adam(learning_rate=sampled_params['learning_rate']), loss='categorical_crossentropy', metrics=['accuracy'])
  H = modelViT.fit(embeddings_train, y_train_filtered, validation_data=(embeddings_val, y_val_filtered), epochs=epochs, batch_size=sampled_params['batch_size'])

  val_probs = modelViT.predict(embeddings_val)
  val_labels_pred = np.argmax(val_probs, axis=1)
  y_val_filtered__ = np.argmax(y_val_filtered, axis=1)
  val_accuracy =accuracy_score(y_val_filtered__, val_labels_pred)

  if val_accuracy > best_validation_accuracy:
    best_validation_accuracy = val_accuracy
    best_hyperparameters = sampled_params

print("Best Hyperparameters:", best_hyperparameters)
print("Best Validation Accuracy:", best_validation_accuracy)

Epoch 1/100
486/486 [==============================] - 1s 2ms/step - loss: 0.8777 - accuracy: 0.7051 - val_loss: 0.8610 - val_accuracy: 0.7113
Epoch 2/100
486/486 [==============================] - 1s 2ms/step - loss: 0.7598 - accuracy: 0.7490 - val_loss: 0.7942 - val_accuracy: 0.7460
Epoch 3/100
486/486 [==============================] - 1s 2ms/step - loss: 0.7425 - accuracy: 0.7597 - val_loss: 0.7612 - val_accuracy: 0.7615
Epoch 4/100
486/486 [==============================] - 1s 2ms/step - loss: 0.7142 - accuracy: 0.7671 - val_loss: 0.8402 - val_accuracy: 0.7434
Epoch 5/100
486/486 [==============================] - 1s 2ms/step - loss: 0.7154 - accuracy: 0.7685 - val_loss: 0.8058 - val_accuracy: 0.7374
Epoch 6/100
486/486 [==============================] - 1s 2ms/step - loss: 0.7017 - accuracy: 0.7708 - val_loss: 0.8530 - val_accuracy: 0.7381
Epoch 7/100
486/486 [==============================] - 1s 2ms/step - loss: 0.6933 - accuracy: 0.7736 - val_loss: 0.8712 - val_accuracy: 0.7316

In [ ]:
best_num_hidden_units = 128
best_dropout_rate = 0.36513715711429373

inputs = Input(shape=(embeddings_train.shape[1],))
x = Dense(best_num_hidden_units, activation='relu')(inputs)
dropout_layer = Dropout(rate=best_dropout_rate)(x)

# Define the output layer (assuming it's a classification task)
predictions = Dense(7, activation='softmax')(dropout_layer)

# Create the functional model
modelViT = Model(inputs, predictions)

In [ ]:
epochs = 10000
batch_size = 64


# Drop learning rate scheduler
def learning_rate_scheduler(epoch, learning_rate):
  if epoch % epochs_drop == 0 and epoch != 0:
    return learning_rate * drop_rate
  else:
    return learning_rate

initial_learning_rate =0.0669615757398038
epochs_drop = 150
drop_rate = 0.95
lr_scheduler = LearningRateScheduler(learning_rate_scheduler)
optimizer = Adam(learning_rate=initial_learning_rate)
# Early Stopping
early_stopping = EarlyStopping(monitor='val_loss', patience=400, restore_best_weights=True)


modelViT.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])
H = modelViT.fit(embeddings_train, y_train_filtered, validation_data=(embeddings_val, y_val_filtered), epochs=epochs, batch_size=batch_size, callbacks=[early_stopping])


Epoch 1/10000
486/486 [==============================] - 1s 2ms/step - loss: 1.1672 - accuracy: 0.5913 - val_loss: 0.9361 - val_accuracy: 0.6590
Epoch 2/10000
486/486 [==============================] - 1s 2ms/step - loss: 1.1306 - accuracy: 0.5978 - val_loss: 0.9719 - val_accuracy: 0.6771
Epoch 3/10000
486/486 [==============================] - 1s 2ms/step - loss: 1.1139 - accuracy: 0.5999 - val_loss: 1.0199 - val_accuracy: 0.6648
Epoch 4/10000
486/486 [==============================] - 1s 2ms/step - loss: 1.1145 - accuracy: 0.5907 - val_loss: 0.9839 - val_accuracy: 0.6765
Epoch 5/10000
486/486 [==============================] - 1s 2ms/step - loss: 1.0948 - accuracy: 0.5954 - val_loss: 0.9243 - val_accuracy: 0.6703
Epoch 6/10000
486/486 [==============================] - 1s 2ms/step - loss: 1.0935 - accuracy: 0.6021 - val_loss: 1.0351 - val_accuracy: 0.6391
Epoch 7/10000
486/486 [==============================] - 1s 2ms/step - loss: 1.1350 - accuracy: 0.5805 - val_loss: 1.0642 - val_ac

In [ ]:
print("Evaluación del modelo...")
y_pred_modelViT_val = modelViT.predict(X_emb_img_val, batch_size=64)
print(classification_report(y_val_filtered.argmax(axis=1), y_pred_modelViT_val.argmax(axis=1)))

y_pred_modelViT_test = modelViT.predict(X_emb_img_test, batch_size=64)
print(classification_report(y_test_filtered.argmax(axis=1), y_pred_modelViT_test.argmax(axis=1)))


Evaluación del modelo...
86/86 [==============================] - 0s 774us/step
              precision    recall  f1-score   support

           0       0.93      0.34      0.49       898
           1       0.89      0.32      0.47       917
           2       0.95      0.61      0.74       909
           3       0.31      0.94      0.46       921
           4       1.00      0.59      0.74        22
           5       0.95      0.70      0.81       922
           6       0.98      0.82      0.90       895

    accuracy                           0.62      5484
   macro avg       0.86      0.62      0.66      5484
weighted avg       0.83      0.62      0.65      5484

101/101 [==============================] - 0s 922us/step
              precision    recall  f1-score   support

           0       0.91      0.31      0.46      1065
           1       0.90      0.35      0.51      1044
           2       0.94      0.61      0.74      1040
           3       0.31      0.92      0.46      

Guardamos los valores obtenidos para los hiperparámetros, para futuras iteraciones.
Los valores obtenidos en este modelo multiclase no son muy buenos, pero nos penaliza la clase unknown, que sería conveniente eliminar.

In [ ]:
Best Hyperparameters: {'learning_rate': 0.013535321123055095, 'batch_size': 32, 'num_hidden_units': 128, 'dropout_rate': 0.22925637991902226}

In [ ]:
modelViT.save('/content/drive/MyDrive/Proyecto Cultivos/deepglobe/model_vit_128_6classes.keras')
modelViT.save('/content/drive/MyDrive/Proyecto Cultivos/deepglobe/model_vit_128_6classes.h5')